In [ ]:
C:\Users\Lawlzee\Documents\Projects\Lawlzee\TAO3\TAO3\TAO3\build.ps1
#!csharp
#i C:\Users\Lawlzee\Documents\Projects\Lawlzee\TAO3\TAO3\TAO3\bin\Feed
#r "nuget:TAO3"
using static TAO3.Prelude;

Build started 2021-06-25 2:09:26 PM.


Project "C:\Users\Lawlzee\Documents\Projects\Lawlzee\TAO3\TAO3\TAO3\TAO3.csproj" on node 1 (Pack target(s)).


GenerateAvaloniaResources:


Skipping target "GenerateAvaloniaResources" because all output files are up-to-date with respect to the input files.


GenerateTargetFrameworkMonikerAttribute:


...

Installed TAO3 1.0.0 from  with content hash yebMl9FoZdCVZKEusRDugZ64zZ2DGkXZw3QqZoKLqXRwAaLZLOby6lW9FtLhsAQ3EpjNi6s0u5AlGXqPRxpq+g==.


Successfully added package 'bin\Debug\TAO3.1.0.0.nupkg' to feed 'bin\Feed'.


Restore sources C:\Users\Lawlzee\Documents\Projects\Lawlzee\TAO3\TAO3\TAO3\bin\Feed

Installed package TAO3 version 1.0.0

Loading extensions from `TAO3.dll`

In [ ]:
#i C:\Users\Lawlzee\Documents\Projects\Lawlzee\TAO3\TAO3\TAO3\bin\Feed
#r "nuget:TAO3"
using static TAO3.Prelude;

In [ ]:
#r nuget:System.Data.SqlClient
#i C:\Users\Lawlzee\Documents\Projects\Lawlzee\TAO3\TAO3\TAO3\bin\Feed
#r "nuget:TAO3"

using static TAO3.Prelude;
using System.Data.SqlClient;

string connectionString = "Server= localhost\\SQLEXPRESS; Database=TestDatabase; Integrated Security=True;";
string schemaQuery = @"SELECT [EventType]
,[SchemaName]
,[ObjectName]
,[ObjectType]
,[EventDate]
,[SystemUser]
,[CurrentUser]
,[OriginalUser]
FROM [TAO3].[SchemaChange]";
string queueName = "MyQueue";
SqlDependency.Start(connectionString);

/*
ALTER DATABASE [DBNAME] SET ENABLE_BROKER WITH ROLLBACK IMMEDIATE
CREATE QUEUE myQueue
CREATE SERVICE myService ON QUEUE myQueue 

([http://schemas.microsoft.com/SQL/Notifications/PostQueryNotification])

*/

/*
CREATE SCHEMA TAO3

CREATE TABLE [TAO3].[SchemaChange]
(
	[EventType] [varchar](100) NULL,
	[SchemaName] [varchar](100) NULL,
	[ObjectName] [varchar](100) NULL,
	[ObjectType] [varchar](100) NULL,
	[EventDate] [datetime] NULL,
	[SystemUser] [varchar](100) NULL,
	[CurrentUser] [varchar](100) NULL,
	[OriginalUser] [varchar](100) NULL
)
GO
CREATE TRIGGER TAO3_SchemaChanged_Trigger
ON DATABASE
FOR DDL_DATABASE_LEVEL_EVENTS
AS
set nocount on
declare @EventType varchar(100)
declare @SchemaName varchar(100)
declare @ObjectName varchar(100)
declare @ObjectType varchar(100)
SELECT 
 @EventType = EVENTDATA().value('(/EVENT_INSTANCE/EventType)[1]','nvarchar(max)')  
,@SchemaName = EVENTDATA().value('(/EVENT_INSTANCE/SchemaName)[1]','nvarchar(max)')  
,@ObjectName = EVENTDATA().value('(/EVENT_INSTANCE/ObjectName)[1]','nvarchar(max)')
,@ObjectType = EVENTDATA().value('(/EVENT_INSTANCE/ObjectType)[1]','nvarchar(max)')   
-- Is the default schema used 
if @SchemaName = ' ' select @SchemaName = default_schema_name from sys.sysusers u join sys.database_principals p 
                        on u.uid = p.principal_id  where u.name = CURRENT_USER
insert into [TAO3].[SchemaChange] 
      select @EventType, @SchemaName, @ObjectName, @ObjectType, getdate(), SUSER_SNAME(), CURRENT_USER, ORIGINAL_LOGIN()  
*/

In [ ]:
AddNotification();

void AddNotification()
{
    using (SqlConnection connection = new SqlConnection(connectionString))
    {
        connection.Open();
        using (SqlCommand command = new SqlCommand(schemaQuery, connection))
        {
            SqlDependency dependency = new SqlDependency(command);
        
            dependency.OnChange += (s, e) => 
            {
                Toast.Notify(e.Source.ToString(), e.Info.ToString(), DateTimeOffset.Now);
                if (e.Info != SqlNotificationInfo.Invalid)
                {
                    AddNotification();
                }
                
            };
    
            //command.Connection.Open();
            command.ExecuteNonQuery();
        }
    }
}

In [ ]:
SqlDependency.Stop(connectionString);

In [ ]:
#!cell MyCell
DateTime a = DateTime.Now;
return a;

In [ ]:
#!run cell MyCell

In [ ]:
return a;

In [ ]:
#!run notebook Test1.dib

In [ ]:
return CSharp;

In [ ]:
#!run notebook Test2.ipynb

In [ ]:
#!powershell

In [ ]:
#!run file "C:\Users\Lawlzee\Desktop\new 1.txt" 

In [ ]:
return new TestClass{
    A = "D",
    B = "kjdj"
};

In [ ]:
using System.IO;
return Path.GetPathRoot("C:\\");
return Path.IsPathFullyQualified("C:\\");
return Path.GetDirectoryName("C:\\");
return Path.GetPathRoot("C:\\djlkhdukj");
return Path.GetDirectoryName("C:\\");
return Directory.Exists("Services");

In [ ]:
return Directory.GetCurrentDirectory();

In [ ]:
using System.CommandLine;
using System.CommandLine.Collections;

return typeof(SymbolSet)
            .GetMethod("Remove", System.Reflection.BindingFlags.NonPublic | System.Reflection.BindingFlags.Instance, null, new Type[] { typeof(ISymbol) }, null)
            !.CreateDelegate<Action<SymbolSet, ISymbol>>();

In [ ]:
#!cell MyCell
return "Hello world!2";

In [ ]:
#!run cell MyCell

In [ ]:
Cells.RemoveCell("MyCell");

In [ ]:
#!in notepad line temp
return temp;

In [ ]:
InputSource.Remove("notepad");

In [ ]:
#!macro CTRL+SHIFT+1
return "Hello world!";

<!DOCTYPE html>
 
 
 
 
 
 
 Macro was registered successfully. Use CTRL+SHIFT+1 to run the macro.

In [ ]:
#!in notepad line lines

In [ ]:
#!out notepad json

using System.Text.RegularExpressions;

string name = null;
string value = null;
List<string> description = new List<string>();

class Row
{
    public string Name {get; set;}
    public string Value {get; set;}
    public string Description {get; set;}
}

List<Row> rows = new List<Row>();

string lastLine = null;

foreach (string line in lines)
{
    if (Regex.IsMatch(line, @"^\d"))
    {
        if (value != null)
        {
            rows.Add(new Row{
                Name = name,
                Value = value,
                Description = string.Join(Environment.NewLine, description.SkipLast(1))
            });

            name = null;
            description = new List<string>();
        }

        name = lastLine;
        value = line;
        description = new List<string>();
    }
    else
    {
        description.Add(line); 
    }
    lastLine = line;
}

return rows;

In [ ]:
#!out notepad line
return rows.Select(row => $@"
            /// <summary>
            /// {row.Description.Replace(Environment.NewLine, Environment.NewLine + "            /// ")}
            /// </summary>
            {row.Name} = {row.Value},");

In [ ]:
return await Clipboard.GetFilesAsync();

index,value
0,C:\Users\Lawlzee\Documents\Projects\Github\dotnet-interactive\tools
1,C:\Users\Lawlzee\Documents\Projects\Github\dotnet-interactive\samples
2,C:\Users\Lawlzee\Documents\Projects\Github\dotnet-interactive\src
